<a href="https://colab.research.google.com/github/ShotaroBaba/NLP_Practice/blob/NLP_smaller_dataset/NLPIMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Firstly, download text dataset.:
# This time, IMDB comment dataset has been used for the experiment.
import os
import gensim
import requests 
from gensim.parsing.preprocessing import stem
from gensim.parsing.preprocessing import strip_punctuation

from gensim.corpora import Dictionary
urlToWiki = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
savedFileName = 'aclImdb_v1.tar.gz'
if not os.path.isfile(savedFileName):
  with open(savedFileName,'wb') as output:
    output.write(requests.get(urlToWiki).content)

In [0]:
import tarfile

# Extract tar.gz file
tarfile.open(savedFileName, "r:gz").extractall()

In [0]:
neg_test = [os.path.join(root,x) for root, _, files in os.walk("aclImdb/test/neg/", topdown=False) for x in files if x.endswith(".txt") ]
pos_test  = [os.path.join(root,x) for root, _, files in os.walk("aclImdb/test/pos/", topdown=False) for x in files if x.endswith(".txt") ]

In [6]:
print(neg_test[:10])

['aclImdb/test/neg/4332_2.txt', 'aclImdb/test/neg/3195_2.txt', 'aclImdb/test/neg/7053_1.txt', 'aclImdb/test/neg/9996_2.txt', 'aclImdb/test/neg/3744_4.txt', 'aclImdb/test/neg/1708_1.txt', 'aclImdb/test/neg/7690_3.txt', 'aclImdb/test/neg/11833_4.txt', 'aclImdb/test/neg/11344_1.txt', 'aclImdb/test/neg/5580_1.txt']


In [0]:
neg_train = [os.path.join(root,x) for root, _, files in os.walk("aclImdb/train/neg/", topdown=False) for x in files if x.endswith(".txt")]
pos_train  =[os.path.join(root,x) for root, _, files in os.walk("aclImdb/train/pos/", topdown=False) for x in files if x.endswith(".txt")]

In [0]:
# Return texts from path
# if the text is too short, then it will omit it. 
def fetch_text(path):
  with open(path) as f:
    text = f.read()
    if len(text.split()) < 50:
      return False
    else:
      return text

In [0]:
# Next, the files are retrieved as the list of texts.
neg_test = list(filter(None, [fetch_text(x) for x in neg_test]))
pos_test = list(filter(None, [fetch_text(x) for x in pos_test]))
neg_train = list(filter(None, [fetch_text(x) for x in neg_train]))
pos_train = list(filter(None, [fetch_text(x) for x in pos_train]))

In [0]:
# Aggregate all lists into one:

all_text = neg_test + pos_test + neg_train + pos_train

In [0]:
# Load another necessary libraries and functions
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])
def lemmatize_sentence(text):
  doc = nlp(text)
  return " ".join([token.lemma_ for token in doc])
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_short, strip_numeric
from collections import Counter
stopwords = gensim.parsing.preprocessing.STOPWORDS

In [0]:
# Pre-process these given texts, by removing stop words and lemmatization.
all_text_processed = [preprocess_string(lemmatize_sentence(x.lower()), 
                                        filters = [strip_tags, strip_short, 
                                                   strip_punctuation, strip_multiple_whitespaces]) for x in all_text]

In [0]:
# Display how one of them is pre-processed:

all_text_processed = [[x for x in str_list if len(x) > 1 and not x in stopwords and not x == "PRON"] for str_list in all_text_processed]

In [14]:
print(all_text_processed[0])

['wow', 'watch', 'night', 'mccool', 'yesterday', 'wow', 'major', 'spoilers', 'like', 'summary', 'plot', 'stupid', 'pointless', 'movie', 'tell', 'anybody', 'watch', 'inflict', 'pain', 'glimpse', 'huge', 'chunk', 'plot', 'randy', 'work', 'bar', 'mccool', 'meet', 'woman', 'jewel', 'convince', 'surprisingly', 'sex', 'boyfriend', 'end', 'try', 'rob', 'kill', 'randy', 'cousin', 'detective', 'scene', 'crime', 'fall', 'jewel', 'mascot', 'stupidity', 'use', 'guy', 'want', 'involve', 'dvd', 'player', 'randy', 'hire', 'hit', 'man', 'kill', 'eventually', 'detective', 'kill', 'boyfriend', 'psycho', 'brother', 'hit', 'man', 'jewel', 'seriously', 'leave', 'hardly', 'sex', 'scene', 'nearly', 'pornographic', 'scene', 'liv', 'tyl', 'jewel', 'use', 'hose', 'flaunt', 'sexuality', 'point', 'movie', 'honest', 'think', 'producer', 'director', 'male', 'urge', 'think', 'absolutely', 'uncalled', 'plain', 'stupid', 'watch', 'movie', 'want', 'plot', 'want', 'character', 'care', 'sexy', 'woman', 'flaunt', 'happen'

In [0]:
# Create dictionary & BOW:
dictionary = Dictionary(all_text_processed)
corpus = [dictionary.doc2bow(x) for x in all_text_processed]

In [0]:
lda_imdb = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=5)

In [17]:
# You can see that the topic is all about movie and drama series.

lda_imdb.show_topics()

[(0,
  '0.027*"film" + 0.012*"good" + 0.011*"movie" + 0.006*"bad" + 0.005*"like" + 0.005*"scene" + 0.004*"look" + 0.004*"action" + 0.004*"plot" + 0.004*"time"'),
 (1,
  '0.050*"movie" + 0.016*"like" + 0.015*"film" + 0.014*"good" + 0.013*"watch" + 0.010*"think" + 0.010*"time" + 0.008*"bad" + 0.007*"great" + 0.006*"character"'),
 (2,
  '0.008*"good" + 0.008*"play" + 0.008*"movie" + 0.007*"film" + 0.006*"great" + 0.005*"character" + 0.005*"time" + 0.005*"role" + 0.005*"star" + 0.005*"like"'),
 (3,
  '0.014*"movie" + 0.007*"like" + 0.007*"good" + 0.007*"film" + 0.006*"man" + 0.006*"time" + 0.006*"love" + 0.006*"life" + 0.005*"story" + 0.005*"character"'),
 (4,
  '0.035*"film" + 0.007*"story" + 0.006*"character" + 0.005*"like" + 0.005*"time" + 0.005*"life" + 0.005*"good" + 0.004*"work" + 0.004*"people" + 0.004*"way"')]

In [0]:
# Count numbers of words in corpus.
def count_words_in_corpus(corpus):
  count_dict = Counter() 
  for word_count in corpus:
    for key, value in word_count:
      count_dict.update({dictionary[key] : value})

  return count_dict

In [0]:
count_of_words = count_words_in_corpus(corpus)

In [0]:
sorted_count = sorted(count_of_words.items(), key = lambda x: x[1], reverse = True)

In [21]:
print("\n".join([str(x) for x in sorted_count[:20]]))

('movie', 101922)
('film', 94644)
('like', 43949)
('good', 40883)
('time', 31142)
('character', 27966)
('watch', 27346)
('bad', 26082)
('story', 25046)
('think', 23103)
('scene', 21149)
('great', 19805)
('look', 19542)
('know', 18990)
('people', 18302)
('way', 17120)
('love', 17003)
('play', 16816)
('come', 16369)
('thing', 16334)


In [0]:
def create_corpus(text_to_be_converted):
  all_text_processed = [preprocess_string(lemmatize_sentence(x.lower()), 
                                          filters = [strip_tags, strip_short, 
                                                    strip_punctuation, strip_multiple_whitespaces, strip_numeric]) for x in text_to_be_converted]
  all_text_processed = [[x for x in str_list if len(x) > 1 and not x in stopwords and not x == "PRON"] for str_list in all_text_processed]
  count_dict = Counter() 
  for word_list in all_text_processed:
    for word in word_list:
      count_dict.update({word: 1})                      
  return count_dict

In [0]:
# Count words for both two sentiment.
neg = neg_test + neg_train
pos = pos_test + pos_train

pos_processed_count = create_corpus(pos)
neg_processed_count = create_corpus(neg)

In [24]:
 print(sorted(pos_processed_count.items(), key = lambda x: x[1], reverse = True)[:10])

[('film', 50194), ('movie', 44137), ('good', 22658), ('like', 19922), ('time', 16134), ('great', 14133), ('story', 14035), ('character', 13813), ('watch', 12635), ('love', 11603)]


In [25]:
 print(sorted(neg_processed_count.items(), key = lambda x: x[1], reverse = True)[:10])

[('movie', 57785), ('film', 44453), ('like', 24027), ('bad', 21515), ('good', 18225), ('time', 15008), ('watch', 14713), ('character', 14153), ('think', 11997), ('look', 11529)]


In [0]:
# References
# Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. 
# The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).